#### 00 The initial steps for filtering dataset for denoising

In [1]:
import doubletdetection
import scanpy as sc
import scipy.sparse as sp
import scanpy.external as sce
import numpy as np
import os



In [2]:

context_path = "/home/sah2p/ondemand/singlecell_data/"
organism = 'Arabidopsis/'
adata = sc.read_h5ad(context_path+organism+"SRP166333.h5ad")

#### 01 Creating a New anndata objects with just raw data

In [6]:
# Create a new AnnData object with the raw counts
adata_raw = sc.AnnData(adata.raw.X.copy() if adata.raw is not None else adata.X.copy())

# Keep all obs columns except the specified ones
columns_to_remove = ['nCount_RNA', 'nFeature_RNA', 'Percent.mt', 'Seurat_clusters']
adata_raw.obs = adata.obs.drop(columns=columns_to_remove, errors='ignore')  # Remove specific columns

# Keep the original gene metadata (var)
adata_raw.var = adata.var.copy()  # Preserve all gene information

# Remove processed data that isn't needed
adata_raw.uns = {}  # Remove unstructured data
adata_raw.obsm = {}  # Remove dimensionality reductions (PCA, UMAP)
adata_raw.obsp = {}  # Remove pairwise distances
adata_raw.varm = {}  # Remove additional variable metadata

# Print summary to confirm
print(adata_raw)

AnnData object with n_obs × n_vars = 16949 × 53678
    obs: 'Orig.ident', 'Celltype', 'Dataset', 'Tissue', 'Organ', 'Condition', 'Genotype', 'Libraries', 'ACE'
    var: 'features'


#### 02 Perform Basic QC

	1.	Identify mitochondrial genes (in plants, these typically start with "ATMG" for Arabidopsis).
	2.	Calculate the percentage of mitochondrial genes per cell.
	3.	Filter out cells with high mitochondrial content.

In [7]:
adata_raw.var["mt"] = adata_raw.var_names.str.startswith("ATMG")
adata_raw.var["pt"] = adata_raw.var_names.str.startswith("ATCG")
        
sc.pp.calculate_qc_metrics(adata_raw, qc_vars=["mt", "pt"], inplace=True, log1p=True)

In [9]:
adata_raw

AnnData object with n_obs × n_vars = 16949 × 53678
    obs: 'Orig.ident', 'Celltype', 'Dataset', 'Tissue', 'Organ', 'Condition', 'Genotype', 'Libraries', 'ACE', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'total_counts_pt', 'log1p_total_counts_pt', 'pct_counts_pt'
    var: 'features', 'mt', 'pt', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts'

In [ ]:
sc.pl.violin(
    adata_raw,
    ["n_genes_by_counts", "total_counts", "pct_counts_mt", "pct_counts_pt"],
    jitter=0.4,
    multi_panel=True,
    save="qc_metrics.png",
    show=False,
)

In [12]:
sc.pl.scatter(adata_raw, "total_counts", "n_genes_by_counts", color="pct_counts_pt",save="qc_total_counts_vs_n_genes_by_counts.png", show=False)


/home/sah2p/ondemand/tools/miniconda3/envs/scanpy/lib/python3.9/site-packages/scanpy/plotting/_utils.py:715: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  color = color[sort]


<Axes: title={'center': 'pct counts pt'}, xlabel='total_counts', ylabel='n_genes_by_counts'>

In [12]:

# Identify mitochondrial and plastid genes
mt_genes = [gene for gene in adata_raw.var_names if gene.startswith("ATMG")]
pt_genes = [gene for gene in adata_raw.var_names if gene.startswith("ATCG")]
# Print the number of detected genes
print(f"Found {len(mt_genes)} mitochondrial genes and {len(pt_genes)} plastid genes.")

# Compute the fraction of mitochondrial gene counts per cell
adata_raw.obs["percent_mt"] = (
    np.sum(adata_raw[:, mt_genes].X, axis=1) / np.sum(adata_raw.X, axis=1)
) * 100

# Compute the fraction of plastid gene counts per cell
adata_raw.obs["percent_pt"] = (
    np.sum(adata_raw[:, pt_genes].X, axis=1) / np.sum(adata_raw.X, axis=1)
) * 100

adata_raw.obs["n_genes_by_counts"] = (adata_raw.X > 0).sum(axis=1)
# Compute total UMI counts per cell (nCount_RNA equivalent)
adata_raw.obs["total_counts"] = adata_raw.X.sum(axis=1)


# Visualize distributions before filtering
sc.pl.violin(adata_raw, ["percent_mt", "percent_pt"], jitter=0.4, log=True, ylabel="Percentage of Counts", save="percent_mtpt.png")
sc.pl.violin(adata_raw,["n_genes_by_counts","total_counts"], jitter=0.4, log=True, ylabel="Counts", save="counts.png")


Q1 = np.percentile(adata_raw.obs["n_genes_by_counts"], 25)  # First quartile
Q3 = np.percentile(adata_raw.obs["n_genes_by_counts"], 75)  # Third quartile
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Step 6: Apply IQR-based filtering
adata_raw = adata_raw[
            (adata_raw.obs['percent_mt'] <= 10) &
            (adata_raw.obs['percent_pt'] <= 5) &
            (adata_raw.obs['n_genes_by_counts'] > lower_bound) &
            (adata_raw.obs['n_genes_by_counts'] < upper_bound) &
            (adata_raw.obs['total_counts'] > 200) &
            (adata_raw.obs['total_counts'] < 10000)]

# Step 7: Print summary after filtering
print(f"Remaining cells after filtering: {adata_raw.n_obs}")

Found 38 mitochondrial genes and 89 plastid genes.


/home/sah2p/ondemand/tools/miniconda3/envs/scanpy/lib/python3.9/site-packages/seaborn/_oldcore.py:1119: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context('mode.use_inf_as_na', True):
/home/sah2p/ondemand/tools/miniconda3/envs/scanpy/lib/python3.9/site-packages/seaborn/_oldcore.py:1119: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context('mode.use_inf_as_na', True):
/home/sah2p/ondemand/tools/miniconda3/envs/scanpy/lib/python3.9/site-packages/seaborn/_oldcore.py:1075: FutureWarning: When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.
  data_subset = grouped_data.get_group(pd_key)


/home/sah2p/ondemand/tools/miniconda3/envs/scanpy/lib/python3.9/site-packages/seaborn/_oldcore.py:1119: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context('mode.use_inf_as_na', True):
/home/sah2p/ondemand/tools/miniconda3/envs/scanpy/lib/python3.9/site-packages/seaborn/_oldcore.py:1119: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context('mode.use_inf_as_na', True):
/home/sah2p/ondemand/tools/miniconda3/envs/scanpy/lib/python3.9/site-packages/seaborn/_oldcore.py:1075: FutureWarning: When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.
  data_subset = grouped_data.get_group(pd_key)


Remaining cells after filtering: 14279


In [13]:
# Visualize distributions before filtering
sc.pl.violin(adata_raw, ["percent_mt", "percent_pt"], jitter=0.4, log=True, ylabel="Percentage of Counts", save="afterpercent_mtpt.png")
sc.pl.violin(adata_raw,["n_genes_by_counts","total_counts"], jitter=0.4, log=True, ylabel="Counts", save="aftercounts.png")


/home/sah2p/ondemand/tools/miniconda3/envs/scanpy/lib/python3.9/site-packages/seaborn/_oldcore.py:1119: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context('mode.use_inf_as_na', True):
/home/sah2p/ondemand/tools/miniconda3/envs/scanpy/lib/python3.9/site-packages/seaborn/_oldcore.py:1119: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context('mode.use_inf_as_na', True):
/home/sah2p/ondemand/tools/miniconda3/envs/scanpy/lib/python3.9/site-packages/seaborn/_oldcore.py:1075: FutureWarning: When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.
  data_subset = grouped_data.get_group(pd_key)


/home/sah2p/ondemand/tools/miniconda3/envs/scanpy/lib/python3.9/site-packages/seaborn/_oldcore.py:1119: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context('mode.use_inf_as_na', True):
/home/sah2p/ondemand/tools/miniconda3/envs/scanpy/lib/python3.9/site-packages/seaborn/_oldcore.py:1119: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context('mode.use_inf_as_na', True):
/home/sah2p/ondemand/tools/miniconda3/envs/scanpy/lib/python3.9/site-packages/seaborn/_oldcore.py:1075: FutureWarning: When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.
  data_subset = grouped_data.get_group(pd_key)


#### 04 Doublet Detection

In [14]:
clf = doubletdetection.BoostClassifier(
    n_iters=10, 
    clustering_algorithm="louvain", 
    standard_scaling=True,
    pseudocount=0.1,
    n_jobs=-1,
)
doublets = clf.fit(adata_raw.X).predict(p_thresh=1e-16, voter_thresh=0.5)
doublet_score = clf.doublet_score()
adata_raw.obs["doublet"] = doublets
adata_raw.obs["doublet_score"] = doublet_score


  0%|          | 0/10 [00:00<?, ?it/s]

/tmp/ipykernel_4170128/3847579300.py:10: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata_raw.obs["doublet"] = doublets


In [15]:
sc.tl.pca(adata_raw, n_comps=50)
sc.pp.neighbors(adata_raw) #Default
# sce.pp.bbknn(adata_raw, batch_key="Orig.ident") # OPTION 2 for neighbors comparison
sc.tl.umap(adata_raw)
sc.pl.umap(adata_raw, color=["doublet", "doublet_score"], save="_doublet.png")
adata_raw = adata_raw[adata_raw.obs['doublet'] == 0, :]
print(f"Remaining cells after filtering: {adata_raw.n_obs}")

Remaining cells after filtering: 14143


In [16]:
sc.pl.pca(adata_raw, color=["doublet", "doublet_score"], save="_doublet.png")

In [24]:
adata_raw.layers['counts'] = adata_raw.X.copy()
        # Normalizing to median total counts
sc.pp.normalize_total(adata_raw)
# Logarithmize the data:
sc.pp.log1p(adata_raw)
s

#Selecting HVG
sc.pp.highly_variable_genes(adata_raw, n_top_genes=6000, batch_key="Orig.ident")
sc.pl.highly_variable_genes(adata_raw, save = "HVG.png")

# TODO(Sania): Add flag to keep all genes
#Filter to keep only highly variable genes
adata_raw = adata_raw[:, adata_raw.var["highly_variable"]]
sc.pp.scale(adata_raw, max_value=10)

/tmp/ipykernel_2962554/2571401706.py:1: ImplicitModificationWarning: Setting element `.layers['counts']` of view, initializing view as actual.
  adata_raw.layers['counts'] = adata_raw.X.copy()


In [25]:
print(f"Remaining cells after filtering: {adata_raw.n_obs}")

Remaining cells after filtering: 14234


In [26]:
sc.tl.pca(adata_raw, n_comps=50)
sc.pl.pca(adata_raw, color="Orig.ident", save="PCA_before_bc.png")
sc.pp.neighbors(adata_raw) #Default
# sce.pp.bbknn(adata_raw, batch_key="Orig.ident") # OPTION 2 for neighbors comparison
sc.tl.umap(adata_raw)
sc.pl.umap(adata_raw, color=["Orig.ident", "Condition"], save="CheckingForBatchEffect.png")

/home/sah2p/ondemand/tools/miniconda3/envs/scanpy/lib/python3.9/site-packages/scanpy/preprocessing/_pca.py:229: ImplicitModificationWarning: Setting element `.obsm['X_pca']` of view, initializing view as actual.
  adata.obsm['X_pca'] = X_pca


/home/sah2p/ondemand/tools/miniconda3/envs/scanpy/lib/python3.9/site-packages/scanpy/plotting/_tools/scatterplots.py:394: UserWarning: No data for colormapping provided via 'c'. Parameters 'cmap' will be ignored
  cax = scatter(


/home/sah2p/ondemand/tools/miniconda3/envs/scanpy/lib/python3.9/site-packages/scanpy/plotting/_tools/scatterplots.py:394: UserWarning: No data for colormapping provided via 'c'. Parameters 'cmap' will be ignored
  cax = scatter(
/home/sah2p/ondemand/tools/miniconda3/envs/scanpy/lib/python3.9/site-packages/scanpy/plotting/_tools/scatterplots.py:394: UserWarning: No data for colormapping provided via 'c'. Parameters 'cmap' will be ignored
  cax = scatter(


### Addressing batch effect

#### BKNN

In [27]:
import scanpy.external as sce

sce.pp.bbknn(adata_raw, batch_key="Orig.ident")  # Adjusts batch effect
sc.tl.umap(adata_raw)
sc.tl.pca(adata_raw, n_comps=50)

# Visualize after correction
sc.pl.pca(adata_raw, color="Orig.ident", save="PCA_after_bc_bknn.png")
sc.pl.umap(adata_raw, color=["Orig.ident", "Condition"], save="AfterBatchEffectCorrection_BKNN.png")



/home/sah2p/ondemand/tools/miniconda3/envs/scanpy/lib/python3.9/site-packages/scanpy/plotting/_tools/scatterplots.py:394: UserWarning: No data for colormapping provided via 'c'. Parameters 'cmap' will be ignored
  cax = scatter(


/home/sah2p/ondemand/tools/miniconda3/envs/scanpy/lib/python3.9/site-packages/scanpy/plotting/_tools/scatterplots.py:394: UserWarning: No data for colormapping provided via 'c'. Parameters 'cmap' will be ignored
  cax = scatter(
/home/sah2p/ondemand/tools/miniconda3/envs/scanpy/lib/python3.9/site-packages/scanpy/plotting/_tools/scatterplots.py:394: UserWarning: No data for colormapping provided via 'c'. Parameters 'cmap' will be ignored
  cax = scatter(


#### Harmony


In [30]:

#  batch column exists in adata.obs
batch_key = "Orig.ident"  # Change this to match your batch column in adata.obs

# Run Harmony
# `adata` is a variable that likely represents an Anndata object, which is a data structure commonly used in single-cell genomics analysis. It typically contains information about gene expression data, cell metadata, and other relevant information from single-cell experiments. In the provided code snippet, `adata` is being used to store and manipulate single-cell data, including running Harmony batch correction to remove batch effects, performing downstream analysis using the corrected data, and saving the corrected data to a file in H5AD format.
# Run Harmony
sce.pp.harmony_integrate(adata_raw, batch_key)
# Save corrected PCA representation
# Use Harmony-corrected PCA for downstream analysis
sc.pp.neighbors(adata_raw, use_rep="X_pca_harmony")
sc.tl.umap(adata_raw)
sc.pl.umap(adata_raw, color=["Orig.ident", "Condition"], save="AfterBatchEffectCorrection_Harmony.png")
sc.tl.leiden(adata_raw)
sc.pl.umap(adata_raw, color=["leiden"], save="LeidenClusters.png")
sc.tl.louvain(adata_raw)
sc.pl.umap(adata_raw, color=["louvain"], save="LouvainClusters.png")
# Save the corrected data
# adata.write_h5ad("your_data_harmony_corrected.h5ad")

2025-02-17 10:02:23,433 - harmonypy - INFO - Computing initial centroids with sklearn.KMeans...
2025-02-17 10:02:34,425 - harmonypy - INFO - sklearn.KMeans initialization complete.
2025-02-17 10:02:34,510 - harmonypy - INFO - Iteration 1 of 10
2025-02-17 10:02:37,486 - harmonypy - INFO - Iteration 2 of 10
2025-02-17 10:02:40,437 - harmonypy - INFO - Iteration 3 of 10
2025-02-17 10:02:43,436 - harmonypy - INFO - Iteration 4 of 10
2025-02-17 10:02:46,413 - harmonypy - INFO - Iteration 5 of 10
2025-02-17 10:02:49,400 - harmonypy - INFO - Iteration 6 of 10
2025-02-17 10:02:52,382 - harmonypy - INFO - Iteration 7 of 10
2025-02-17 10:02:55,380 - harmonypy - INFO - Iteration 8 of 10
2025-02-17 10:02:57,743 - harmonypy - INFO - Iteration 9 of 10
2025-02-17 10:03:00,655 - harmonypy - INFO - Iteration 10 of 10
2025-02-17 10:03:02,810 - harmonypy - INFO - Converged after 10 iterations


/home/sah2p/ondemand/tools/miniconda3/envs/scanpy/lib/python3.9/site-packages/scanpy/plotting/_tools/scatterplots.py:394: UserWarning: No data for colormapping provided via 'c'. Parameters 'cmap' will be ignored
  cax = scatter(
/home/sah2p/ondemand/tools/miniconda3/envs/scanpy/lib/python3.9/site-packages/scanpy/plotting/_tools/scatterplots.py:394: UserWarning: No data for colormapping provided via 'c'. Parameters 'cmap' will be ignored
  cax = scatter(


/home/sah2p/ondemand/tools/miniconda3/envs/scanpy/lib/python3.9/site-packages/scanpy/plotting/_tools/scatterplots.py:394: UserWarning: No data for colormapping provided via 'c'. Parameters 'cmap' will be ignored
  cax = scatter(


/home/sah2p/ondemand/tools/miniconda3/envs/scanpy/lib/python3.9/site-packages/scanpy/plotting/_tools/scatterplots.py:394: UserWarning: No data for colormapping provided via 'c'. Parameters 'cmap' will be ignored
  cax = scatter(


#### Scanorama

In [37]:
sce.pp.scanorama_integrate(adata_raw, batch_key, verbose=1)
sc.pp.neighbors(adata_raw, use_rep="X_scanorama")
sc.tl.umap(adata_raw)
sc.pl.umap(adata_raw, color=["Orig.ident", "Condition"], save="AfterBatchEffectCorrection_Scanorama.png")
sc.tl.leiden(adata_raw)
sc.pl.umap(adata_raw, color=["leiden"], save="LeidenClusters_Scanorama.png")
sc.tl.louvain(adata_raw)
sc.pl.umap(adata_raw, color=["louvain"], save="LouvainClusters_Scanorama.png")

Processing datasets SRX8089019 <=> SRX8089020
Processing datasets SRX8089020 <=> SRX8089021


/home/sah2p/ondemand/tools/miniconda3/envs/scanpy/lib/python3.9/site-packages/scanpy/plotting/_tools/scatterplots.py:394: UserWarning: No data for colormapping provided via 'c'. Parameters 'cmap' will be ignored
  cax = scatter(
/home/sah2p/ondemand/tools/miniconda3/envs/scanpy/lib/python3.9/site-packages/scanpy/plotting/_tools/scatterplots.py:394: UserWarning: No data for colormapping provided via 'c'. Parameters 'cmap' will be ignored
  cax = scatter(


/home/sah2p/ondemand/tools/miniconda3/envs/scanpy/lib/python3.9/site-packages/scanpy/plotting/_tools/scatterplots.py:394: UserWarning: No data for colormapping provided via 'c'. Parameters 'cmap' will be ignored
  cax = scatter(


/home/sah2p/ondemand/tools/miniconda3/envs/scanpy/lib/python3.9/site-packages/scanpy/plotting/_tools/scatterplots.py:394: UserWarning: No data for colormapping provided via 'c'. Parameters 'cmap' will be ignored
  cax = scatter(


#### Combat

In [46]:
sc.pp.combat(adata_raw, key="Orig.ident")
sc.pp.neighbors(adata_raw)
sc.tl.umap(adata_raw)
sc.tl.pca(adata_raw, n_comps=50)

# Visualize after correction
sc.pl.pca(adata_raw, color="Orig.ident", save="PCA_after_bc_COMBAT.png")
sc.pl.umap(adata_raw, color=["Orig.ident", "Condition"], save="AfterBatchEffectCorrection_COMBAT.png")


/home/sah2p/ondemand/tools/miniconda3/envs/scanpy/lib/python3.9/site-packages/scanpy/preprocessing/_combat.py:340: RuntimeWarning: divide by zero encountered in divide
  (abs(g_new - g_old) / g_old).max(), (abs(d_new - d_old) / d_old).max()


/home/sah2p/ondemand/tools/miniconda3/envs/scanpy/lib/python3.9/site-packages/scanpy/plotting/_tools/scatterplots.py:394: UserWarning: No data for colormapping provided via 'c'. Parameters 'cmap' will be ignored
  cax = scatter(


/home/sah2p/ondemand/tools/miniconda3/envs/scanpy/lib/python3.9/site-packages/scanpy/plotting/_tools/scatterplots.py:394: UserWarning: No data for colormapping provided via 'c'. Parameters 'cmap' will be ignored
  cax = scatter(
/home/sah2p/ondemand/tools/miniconda3/envs/scanpy/lib/python3.9/site-packages/scanpy/plotting/_tools/scatterplots.py:394: UserWarning: No data for colormapping provided via 'c'. Parameters 'cmap' will be ignored
  cax = scatter(


In [45]:
adata_raw


AnnData object with n_obs × n_vars = 14234 × 6000
    obs: 'Orig.ident', 'Celltype', 'Dataset', 'Tissue', 'Organ', 'Condition', 'Genotype', 'Libraries', 'ACE', 'percent_mt', 'percent_pt', 'n_genes_by_counts', 'total_counts', 'doublet', 'doublet_score', 'leiden', 'louvain'
    var: 'features', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'highly_variable_nbatches', 'highly_variable_intersection'
    uns: 'pca', 'neighbors', 'umap', 'log1p', 'hvg', 'Orig.ident_colors', 'Condition_colors', 'leiden', 'leiden_colors', 'louvain', 'louvain_colors'
    obsm: 'X_pca', 'X_umap', 'X_pca_harmony', 'X_scanorama'
    varm: 'PCs'
    layers: 'counts'
    obsp: 'distances', 'connectivities'

In [81]:
adata_raw

AnnData object with n_obs × n_vars = 16333 × 53678
    obs: 'Orig.ident', 'Celltype', 'Dataset', 'Tissue', 'Organ', 'Condition', 'Genotype', 'Libraries', 'ACE', 'percent_mt', 'percent_pt', 'n_genes_by_counts', 'leiden'
    var: 'features'
    uns: 'pca', 'neighbors', 'umap', 'Orig.ident_colors', 'Condition_colors', 'Tissue_colors', 'Genotype_colors', 'leiden'
    obsm: 'X_pca', 'X_umap', 'X_pca_harmony', 'X_scanorama'
    varm: 'PCs'
    obsp: 'distances', 'connectivities'